# 2. Logistic Regression

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./train_MNIST_logistic_regression.csv')

$X=\begin{pmatrix} x_1 \\ \vdots \\ x_n \end{pmatrix}$

In [3]:
# append 'Intercept' column
Xpred = list(data.columns)
Xpred.remove('label')
Xcols = ['Intercept'] + Xpred

n = data.shape[0]
p = len(Xpred)
data = data.assign(Intercept=np.ones(n))

In [ ]:
X = data[Xcols]
y = data[['label']] == 2

### Coefficient
$\beta'=\beta-\eta J(\beta)$

In [ ]:
eta = 0.005
beta = np.zeros((1 + p,))

for epoch in range(10):
    J_beta = 0
    for i in range(n):
        xi = X.loc[i]
        # yi = float(y.loc[i])
        # J_beta = J_beta + np.dot(yi - 1 / (1 + np.exp(-np.dot(beta.T, xi))), xi)
        yi = 2 * float(y.loc[i]) - 1
        J_beta = J_beta + np.dot(yi, xi) / (1 + np.exp(np.dot(np.dot(-yi, beta.T), xi)))
    beta = beta - eta * J_beta

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


### Logit
$p(X)=1-\frac{1}{1+e^{X\beta}}$

In [ ]:
result = 1 - 1 / (1 + np.exp(np.dot(X, beta)))
pred = result < 0.5

### Test

In [ ]:
test_data = pd.read_csv('./test_MNIST_logistic_regression.csv', names=Xpred)
test_data = test_data.assign(Intercept=np.ones(test_data.shape[0]))

In [ ]:
test_result = 1 - 1 / (1 + np.exp(np.dot(test_data, beta)))
test_result

### Confusion Matrix

In [ ]:
confusion_list = list(zip(list(y.label), list(pred)))
cmatrix = [
    [confusion_list.count((False, False)), confusion_list.count((False, True))],
    [confusion_list.count((True, False)), confusion_list.count((True, True))]
]

In [ ]:
pd.DataFrame([
    [cmatrix[0][0], cmatrix[0][1], sum(cmatrix[0])],
    [cmatrix[1][0], cmatrix[1][1], sum(cmatrix[1])],
    list(np.sum(cmatrix, axis=0)) + [np.sum(cmatrix)]],
    columns=[['Confusion Matrix'] * 3, ['True class'] * 3, [0, 2, 'Total']],
    index=[['Predicted class'] * 3, [0, 2, 'Total']]
)

### ROC curve

In [ ]:
pt = []
for p in np.arange(0.1, 1.0, 0.1):
    pred = result < p
    clist = list(zip(list(y.label), list(pred)))
    TPR = clist.count((False, False)) / list(y.label).count(False)
    FPR = clist.count((False, True)) / list(pred).count(True)
    pt.append((TPR, FPR))
pt